In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import my_common as mc
import extract_local_str as els
import scipy.constants as scc
import os

In [3]:
def read_mutiple_xyz_orca(file):
    """
    read multiple frames in output xyz of lammps, 
    input: dump_file, !!!! now change to 'custum'
    output: result (index,type_atoms, coors) and t 
    """
    f = open(file)
    lft = list(f)
    f.close()
    lt=[]
    t=[]
    natom = int(lft[0].split()[0])
    for il in range(len(lft)):
        if 'Coordinates' in lft[il]:
            lt.append(il)
#             t.append(lft[il+1].split()[0])

    def read_lf(lf):
        coors = np.zeros([natom,3])
        type_atom = []
        index = []
        l=0
        
        for ia in lf[1:1+natom]:
            coors[l,:] = np.array(ia.split()[1:4:1]).astype('float')
#             coors[l,:] = coors[l,:] - np.array([xlo,ylo,zlo])
            type_atom.append((ia.split()[0]))
            l+=1

        type_atom = np.array(type_atom)
        
        return type_atom,coors

    # lf=[]
    result = []
    for it in range(len(lt)):
        if it==len(lt)-1:
    #         lf.append(lft[lt[it]:])
            result.append(read_lf(lft[lt[it]:]))
        else:
    #         lf.append(lft[lt[it]:lt[it+1]-1])
            result.append(read_lf(lft[lt[it]:lt[it+1]]))
    
#     t=np.array(t)
    return result

In [4]:
def read_info(file):
    f=open(file,'r')
    L = f.readlines()
    f.close()
    idx_atom = [int(x) for x in L[0].split()]
    # ebond_r0,ebond_k = [float(x) for x in L[1].split()[:2]]
    # energy_loc, strain_loc = [float(x) for x in L[2].split()[:2]]
    
    bond_info = np.loadtxt(file,skiprows=4)
    return idx_atom, bond_info

In [5]:
def collect_data0(location,ia_list,save_file):
    """
    collect raw data
    """
    is_broken_all = [] 
    type_bb = []
    tmp = pd.DataFrame()
    is_trigger = []
    dist_bond = []

    for ia in ia_list: # 
        for ib in range(100):
            # print(ia,ib)
            if os.path.isfile('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib)):
                idx_atom, bond_info = read_info('{}/ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(location,ia,ia,ib))
                result = read_mutiple_xyz_orca('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib))
                inputs = pd.read_csv('{}/ia{}/elstr{}_{}_xtb/soap418.csv'.format(location,ia,ia,ib),header=None)
                coors0 = np.mean(result[0][1][idx_atom],axis=0)
                is_broken = []
                for ibond in range(len(bond_info)): # range(len(bond_info))
                    coors_atom = result[-1][1][bond_info[ibond,2:4].astype(int)]
                    coors_atom0 = result[0][1][bond_info[ibond,2:4].astype(int)]
                    dist_tmp = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5
            #         dist1.append(dist_tmp)
                    if dist_tmp>bond_info[ibond,-3]*1.5:
                        is_broken.append(1)
                        type_bb.append(int(bond_info[ibond,1]))
                        if (type_bb[-1] == 2) or (type_bb[-1] == 6):
                            coors1 = np.mean(coors_atom0,axis=0)
                            dist_core = np.sum((coors1-coors0)**2)**0.5
                            dist_bond.append(dist_core)
                            if (bond_info[ibond,2:4] == idx_atom).all():
                                is_trigger.append(1)
                            else:
                                is_trigger.append(0)
                    else:
                        is_broken.append(0)
                is_broken = np.array(is_broken)
                is_broken_all.append(is_broken)

                inputs.insert(0,'bond_e',bond_info[:,-2])
                inputs.insert(0,'bond_l',bond_info[:,-1])
                inputs.insert(0,'atom2',bond_info[:,5])
                inputs.insert(0,'atom1',bond_info[:,4])
                inputs.insert(0,'ia',np.repeat(ia,len(bond_info)))
                inputs.insert(0,'output',is_broken)
                inputs.insert(1,'bond_type',bond_info[:,1].astype(int))
                tmp = pd.concat([tmp,inputs[(inputs['bond_type']==2) | (inputs['bond_type']==6)]])

    tmp.to_csv('{}'.format(save_file),index=False)
    type_bb = np.array(type_bb)

In [9]:
def collect_data1(location,ia_list,save_file):
    """
    collect data, remove data from local structures 
    in which BB happened near the QM/MM interface
    """
    class StopLookingForThings(Exception): pass

    is_broken_all = [] 
    type_bb = []
    tmp = pd.DataFrame()
    is_trigger = []
    dist_bond = []
    QM = 0
    for ia in ia_list: # 
        for ib in range(100):
            try:
                if os.path.isfile('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib)):
                    QM = QM+1
                    idx_atom, bond_info = read_info('{}/ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(location,ia,ia,ib))
                    result = read_mutiple_xyz_orca('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib))
                    inputs = pd.read_csv('{}/ia{}/elstr{}_{}_xtb/soap418.csv'.format(location,ia,ia,ib),header=None)
                    coors0 = np.mean(result[0][1][idx_atom],axis=0)
                    is_broken = []
                    for ibond in range(len(bond_info)): # range(len(bond_info))
                        coors_atom = result[-1][1][bond_info[ibond,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info[ibond,2:4].astype(int)]
                        dist_tmp = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5
                #         dist1.append(dist_tmp)
                        if dist_tmp>bond_info[ibond,-3]*1.5:
                            is_broken.append(1)
                            type_bb.append(int(bond_info[ibond,1]))
                            if (type_bb[-1] == 2) or (type_bb[-1] == 6):
                                coors1 = np.mean(coors_atom0,axis=0)
                                dist_core = np.sum((coors1-coors0)**2)**0.5
                                dist_bond.append(dist_core)
                                if (bond_info[ibond,2:4] == idx_atom).all():
                                    is_trigger.append(1)
                                else:
                                    is_trigger.append(0)
                                if dist_core>7:
                                    raise StopLookingForThings()
                        else:
                            is_broken.append(0)
                    is_broken = np.array(is_broken)
                    is_broken_all.append(is_broken)

                    inputs.insert(0,'bond_e',bond_info[:,-2])
                    inputs.insert(0,'bond_l',bond_info[:,-1])
                    inputs.insert(0,'atom2',bond_info[:,5])
                    inputs.insert(0,'atom1',bond_info[:,4])
                    inputs.insert(0,'ia',np.repeat(ia,len(bond_info)))
                    inputs.insert(0,'output',is_broken)
                    inputs.insert(1,'bond_type',bond_info[:,1].astype(int))
                    tmp = pd.concat([tmp,inputs[(inputs['bond_type']==2) | (inputs['bond_type']==6)]])
            except StopLookingForThings:
                pass 
    tmp.to_csv('{}'.format(save_file),index=False)
    type_bb = np.array(type_bb)
    print(QM,len(tmp))

In [8]:
def collect_data2(location,ia_list,save_file):
    """
    collect data, remove data + relabel output
    """
    class StopLookingForThings(Exception): pass

    is_broken_all = [] 
    type_bb = []
    tmp = pd.DataFrame()
    is_trigger = []
    dist_bond = []

    for ia in ia_list: # 
        for ib in range(100):
            try:
                if os.path.isfile('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib)):
                    idx_atom, bond_info = read_info('{}/ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(location,ia,ia,ib))
                    result = read_mutiple_xyz_orca('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib))
                    inputs = pd.read_csv('{}/ia{}/elstr{}_{}_xtb/soap418.csv'.format(location,ia,ia,ib),header=None)
                    coors0 = np.mean(result[0][1][idx_atom],axis=0)
                    
                    is_broken = np.zeros(len(bond_info))
                    idx_broken = []
                    for ibond in range(len(bond_info)): # range(len(bond_info))
                        coors_atom = result[-1][1][bond_info[ibond,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info[ibond,2:4].astype(int)]
                        dist_tmp = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5
                #         dist1.append(dist_tmp)
                        if dist_tmp>bond_info[ibond,-3]*1.5:
                            # is_broken[ibond] = 1
                            idx_broken.append(ibond)
                            type_bb.append(int(bond_info[ibond,1]))
                            if (type_bb[-1] == 2) or (type_bb[-1] == 6):
                                coors1 = np.mean(coors_atom0,axis=0)
                                dist_core = np.sum((coors1-coors0)**2)**0.5
                                dist_bond.append(dist_core)
                                # find the adjacent bond
                                idx_atom_tmp1 = bond_info[ibond,2]
                                idx_atom_tmp2 = bond_info[ibond,3]
                                for jbond in range(len(bond_info)):
                                    if ((idx_atom_tmp1 in bond_info[jbond,2:4]) | (idx_atom_tmp2 in bond_info[jbond,2:4])) & (bond_info[jbond,1] == bond_info[ibond,1]):
                                        if jbond != ibond:
                                            idx_broken.append(jbond)
                                            # print('find',bond_info[ibond,1:4],bond_info[jbond,1:4])
                                # idx_broken.append(idx_bond_adjacent)
                                if (bond_info[ibond,2:4] == idx_atom).all():
                                    is_trigger.append(1)
                                else:
                                    is_trigger.append(0)
                                if dist_core>7:
                                    raise StopLookingForThings()

                    idx_broken = np.array(idx_broken)
                    if len(idx_broken)>0:
                        is_broken[idx_broken]=1
                    
                    is_broken_all.append(is_broken)

                    inputs.insert(0,'bond_e',bond_info[:,-2])
                    inputs.insert(0,'bond_l',bond_info[:,-1])
                    inputs.insert(0,'atom2',bond_info[:,5])
                    inputs.insert(0,'atom1',bond_info[:,4])
                    inputs.insert(0,'ia',np.repeat(ia,len(bond_info)))
                    inputs.insert(0,'output',is_broken)
                    inputs.insert(1,'bond_type',bond_info[:,1].astype(int))
                    tmp = pd.concat([tmp,inputs[(inputs['bond_type']==2) | (inputs['bond_type']==6)]])
            except StopLookingForThings:
                pass 

    tmp.to_csv('{}'.format(save_file),index=False)
    type_bb = np.array(type_bb)

In [28]:
def collect_data2d5(location,ia_list,save_file):
    """
    collect data, relabeling
    """
    class StopLookingForThings(Exception): pass

    is_broken_all = [] 
    type_bb = []
    tmp = pd.DataFrame()
    is_trigger = []
    dist_bond = []

    for ia in ia_list: # 
        for ib in range(100):
            # try:
            if os.path.isfile('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib)):
                idx_atom, bond_info = read_info('{}/ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(location,ia,ia,ib))
                result = read_mutiple_xyz_orca('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib))
                inputs = pd.read_csv('{}/ia{}/elstr{}_{}_xtb/soap418.csv'.format(location,ia,ia,ib),header=None)
                coors0 = np.mean(result[0][1][idx_atom],axis=0)
                
                is_broken = np.zeros(len(bond_info))
                idx_broken = []
                for ibond in range(len(bond_info)): # range(len(bond_info))
                    coors_atom = result[-1][1][bond_info[ibond,2:4].astype(int)]
                    coors_atom0 = result[0][1][bond_info[ibond,2:4].astype(int)]
                    dist_tmp = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5
            #         dist1.append(dist_tmp)
                    if dist_tmp>bond_info[ibond,-3]*1.5:
                        # is_broken[ibond] = 1
                        idx_broken.append(ibond)
                        type_bb.append(int(bond_info[ibond,1]))
                        if (type_bb[-1] == 2) or (type_bb[-1] == 6):
                            coors1 = np.mean(coors_atom0,axis=0)
                            dist_core = np.sum((coors1-coors0)**2)**0.5
                            dist_bond.append(dist_core)
                            # find the adjacent bond
                            idx_atom_tmp1 = bond_info[ibond,2]
                            idx_atom_tmp2 = bond_info[ibond,3]
                            for jbond in range(len(bond_info)):
                                if ((idx_atom_tmp1 in bond_info[jbond,2:4]) | (idx_atom_tmp2 in bond_info[jbond,2:4])) & (bond_info[jbond,1] == bond_info[ibond,1]):
                                    if jbond != ibond:
                                        idx_broken.append(jbond)
                                        # print('find',bond_info[ibond,1:4],bond_info[jbond,1:4])
                            # idx_broken.append(idx_bond_adjacent)
                            if (bond_info[ibond,2:4] == idx_atom).all():
                                is_trigger.append(1)
                            else:
                                is_trigger.append(0)
                            # if dist_core>7:
                            #     raise StopLookingForThings()

                idx_broken = np.array(idx_broken)
                if len(idx_broken)>0:
                    is_broken[idx_broken]=1
                
                is_broken_all.append(is_broken)

                inputs.insert(0,'bond_e',bond_info[:,-2])
                inputs.insert(0,'bond_l',bond_info[:,-1])
                inputs.insert(0,'atom2',bond_info[:,5])
                inputs.insert(0,'atom1',bond_info[:,4])
                inputs.insert(0,'ia',np.repeat(ia,len(bond_info)))
                inputs.insert(0,'output',is_broken)
                inputs.insert(1,'bond_type',bond_info[:,1].astype(int))
                tmp = pd.concat([tmp,inputs[(inputs['bond_type']==2) | (inputs['bond_type']==6)]])
            # except StopLookingForThings:
            #     pass 

    tmp.to_csv('{}'.format(save_file),index=False)
    type_bb = np.array(type_bb)

In [75]:
def collect_data3(location,ia_list,save_file):
    """
    collect data, relabeling
    """
    class StopLookingForThings(Exception): pass

    is_broken_all = [] 
    tmp = pd.DataFrame()
    is_trigger = []
    dist_bond = []

    result_oLE = [] # output, length, energy

    for ia in ia_list: # 
        for ib in range(100):
            # try:
            if os.path.isfile('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib)):
                idx_atom, bond_info = read_info('{}/ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(location,ia,ia,ib))
                result = read_mutiple_xyz_orca('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib))
                # inputs = pd.read_csv('{}/ia{}/elstr{}_{}_xtb/soap418.csv'.format(location,ia,ia,ib),header=None)
                coors0 = np.mean(result[0][1][idx_atom],axis=0)
                
                bond_info_26 = bond_info[(bond_info[:,1]==2) | (bond_info[:,1]==6)]

                is_broken = np.zeros(len(bond_info_26))

                for ibond in range(len(bond_info_26)): 
                    
                    idx_atom_tmp1 = bond_info_26[ibond,2]
                    idx_atom_tmp2 = bond_info_26[ibond,3]

                    for jbond in range(len(bond_info_26)):
                        if ((idx_atom_tmp1 in bond_info_26[jbond,2:4]) | (idx_atom_tmp2 in bond_info_26[jbond,2:4])) & (bond_info_26[jbond,1] == bond_info_26[ibond,1]) & (jbond != ibond):
                            idx_neighbor = jbond
                            break
                        else:
                            idx_neighbor = -1

                    if idx_neighbor>=0:
                        coors_atom = result[-1][1][bond_info_26[ibond,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info_26[ibond,2:4].astype(int)]
                        dist_tmp_i = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

                        coors_atom = result[-1][1][bond_info_26[idx_neighbor,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info_26[idx_neighbor,2:4].astype(int)]
                        dist_tmp_j = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

                        if (dist_tmp_i>bond_info_26[ibond,-3]*1.5) | (dist_tmp_j>bond_info_26[idx_neighbor,-3]*1.5):
                            output = 1
                            # coors1 = np.mean(coors_atom0,axis=0)
                            # dist_core = np.sum((coors1-coors0)**2)**0.5
                            # dist_bond.append(dist_core)
                        else:
                            output = 0
                        
                        result_oLE.append([output,bond_info_26[ibond,-1],bond_info_26[idx_neighbor,-1],bond_info_26[ibond,-2],bond_info_26[idx_neighbor,-2]])

                        inputs = {'output':[output],'ia':[bond_info_26[ibond,1]],'atom1':[bond_info_26[ibond,4]],'atom2':[bond_info_26[ibond,5]],
                                    'atom3':[bond_info_26[idx_neighbor,4]],'atom4':[bond_info_26[idx_neighbor,5]],
                                    'length1':[bond_info_26[ibond,-1]],'length2':[bond_info_26[idx_neighbor,-1]],
                                    'E1':[bond_info_26[ibond,-2]],'E2':[bond_info_26[idx_neighbor,-2]],
                                    'CG_length':[bond_info_26[ibond,-1]+bond_info_26[idx_neighbor,-1]],
                                    'CG_E':[bond_info_26[ibond,-2]+bond_info_26[idx_neighbor,-2]]}
                        tmp = pd.concat([tmp,pd.DataFrame(data=inputs)])

    tmp.to_csv('{}'.format(save_file),index=False)
    # type_bb = np.array(type_bb)
    return result_oLE

In [80]:
def collect_data4(location,ia_list,save_file):
    """
    collect data, CG soap and output
    """
    class StopLookingForThings(Exception): pass

    is_broken_all = [] 
    tmp = []
    is_trigger = []
    dist_bond = []

    # result_oLE = [] # output, length, energy

    for ia in ia_list: # 
        for ib in range(100):
            # try:
            if os.path.isfile('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib)):
                idx_atom, bond_info = read_info('{}/ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(location,ia,ia,ib))
                result = read_mutiple_xyz_orca('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib))
                soap = pd.read_csv('{}/ia{}/elstr{}_{}_xtb/soap418.csv'.format(location,ia,ia,ib),header=None).to_numpy()
                coors0 = np.mean(result[0][1][idx_atom],axis=0)

                idx_26b = np.squeeze(np.argwhere((bond_info[:,1]==2) | (bond_info[:,1]==6)))
                bond_info_26 = bond_info[idx_26b]
                is_broken = np.zeros(len(bond_info_26))

                for ibond in range(len(bond_info_26)): 
                    
                    idx_atom_tmp1 = bond_info_26[ibond,2]
                    idx_atom_tmp2 = bond_info_26[ibond,3]

                    for jbond in range(len(bond_info_26)):
                        if ((idx_atom_tmp1 in bond_info_26[jbond,2:4]) | (idx_atom_tmp2 in bond_info_26[jbond,2:4])) & (bond_info_26[jbond,1] == bond_info_26[ibond,1]) & (jbond != ibond):
                            idx_neighbor = jbond
                            break
                        else:
                            idx_neighbor = -1

                    if idx_neighbor>=0:
                        coors_atom = result[-1][1][bond_info_26[ibond,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info_26[ibond,2:4].astype(int)]
                        dist_tmp_i = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

                        coors_atom = result[-1][1][bond_info_26[idx_neighbor,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info_26[idx_neighbor,2:4].astype(int)]
                        dist_tmp_j = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

                        if (dist_tmp_i>bond_info_26[ibond,-3]*1.5) | (dist_tmp_j>bond_info_26[idx_neighbor,-3]*1.5):
                            output = 1
                            # coors1 = np.mean(coors_atom0,axis=0)
                            # dist_core = np.sum((coors1-coors0)**2)**0.5
                            # dist_bond.append(dist_core)
                        else:
                            output = 0
                        input = np.concatenate(([output,bond_info_26[ibond,1],bond_info_26[ibond,4],bond_info_26[ibond,5],bond_info_26[idx_neighbor,4],
                        bond_info_26[idx_neighbor,5],bond_info_26[ibond,-1],bond_info_26[idx_neighbor,-1],bond_info_26[ibond,-2],bond_info_26[idx_neighbor,-2]],
                        soap[idx_26b[ibond]]+soap[idx_26b[idx_neighbor]])) # some might be counted twice 
                        # print(ib,input.shape)
                        
                        tmp.append(input)
    tmp = pd.DataFrame(np.array(tmp))
    tmp.to_csv('{}'.format(save_file),index=False)
    # type_bb = np.array(type_bb)
    # return result_oLE

In [8]:
def collect_data4ns(location,ia_list,save_file):
    """
    collect data, CG soap and output,
    4ns input: summing SOAP vectors
    notes: compared to 4, 4ns distinguish the common atom in the middle and dont overcount it 
    """
    class StopLookingForThings(Exception): pass

    is_broken_all = [] 
    tmp = []
    is_trigger = []
    dist_bond = []

    # result_oLE = [] # output, length, energy

    for ia in ia_list: # 
        for ib in range(100):
            # try:
            if os.path.isfile('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib)):
                idx_atom, bond_info = read_info('{}/ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(location,ia,ia,ib))
                result = read_mutiple_xyz_orca('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib))
                soap = pd.read_csv('{}/ia{}/elstr{}_{}_xtb/soap418.csv'.format(location,ia,ia,ib),header=None).to_numpy()
                coors0 = np.mean(result[0][1][idx_atom],axis=0)

                idx_26b = np.squeeze(np.argwhere((bond_info[:,1]==2) | (bond_info[:,1]==6)))
                bond_info_26 = bond_info[idx_26b]
                is_broken = np.zeros(len(bond_info_26))

                for ibond in range(len(bond_info_26)): 
                    
                    idx_atom_tmp1 = bond_info_26[ibond,2]
                    idx_atom_tmp2 = bond_info_26[ibond,3]

                    for jbond in range(len(bond_info_26)):
                        if ((idx_atom_tmp1 in bond_info_26[jbond,2:4])) & (bond_info_26[jbond,1] == bond_info_26[ibond,1]) & (jbond != ibond):
                            idx_neighbor = jbond
                            if idx_atom_tmp1 == bond_info_26[jbond,2]:
                                common_at = [0,0] # which of the two atoms is the common atom
                            else:
                                common_at = [0,1]
                            break
                        elif (idx_atom_tmp2 in bond_info_26[jbond,2:4]) & (bond_info_26[jbond,1] == bond_info_26[ibond,1]) & (jbond != ibond):
                            idx_neighbor = jbond
                            if idx_atom_tmp2 == bond_info_26[jbond,2]:
                                common_at = [1,0] # which of the two atoms is the common atom
                            else:
                                common_at = [1,1]
                            break
                        else:
                            idx_neighbor = -1

                    if idx_neighbor>=0:
                        coors_atom = result[-1][1][bond_info_26[ibond,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info_26[ibond,2:4].astype(int)]
                        dist_tmp_i = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

                        coors_atom = result[-1][1][bond_info_26[idx_neighbor,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info_26[idx_neighbor,2:4].astype(int)]
                        dist_tmp_j = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

                        if (dist_tmp_i>bond_info_26[ibond,-3]*1.5) | (dist_tmp_j>bond_info_26[idx_neighbor,-3]*1.5):
                            output = 1
                            # coors1 = np.mean(coors_atom0,axis=0)
                            # dist_core = np.sum((coors1-coors0)**2)**0.5
                            # dist_bond.append(dist_core)
                        else:
                            output = 0
                        nsoap = int(soap.shape[1]/2)
                        input = np.concatenate(([output,bond_info_26[ibond,1],bond_info_26[ibond,4],bond_info_26[ibond,5],bond_info_26[idx_neighbor,4],
                        bond_info_26[idx_neighbor,5],bond_info_26[ibond,-1],bond_info_26[idx_neighbor,-1],bond_info_26[ibond,-2],bond_info_26[idx_neighbor,-2]],
                        soap[idx_26b[ibond],nsoap*common_at[0]:nsoap*(common_at[0]+1)]+
                        soap[idx_26b[ibond],nsoap*(1-common_at[0]):nsoap*(2-common_at[0])]+
                        soap[idx_26b[idx_neighbor],nsoap*(1-common_at[1]):nsoap*(2-common_at[1])]))
                        # print(ib,input.shape)
                        
                        tmp.append(input)
    tmp = pd.DataFrame(np.array(tmp))
    tmp.to_csv('{}'.format(save_file),index=False)

In [12]:
def collect_data4nc(location,ia_list,save_file):
    """
    collect data, CG soap and output,
    4nc input: concatinating the SOAP vectors together
    """
    class StopLookingForThings(Exception): pass

    is_broken_all = [] 
    tmp = []
    is_trigger = []
    dist_bond = []

    # result_oLE = [] # output, length, energy

    for ia in ia_list: # 
        for ib in range(100):
            # try:
            if os.path.isfile('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib)):
                idx_atom, bond_info = read_info('{}/ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(location,ia,ia,ib))
                result = read_mutiple_xyz_orca('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib))
                soap = pd.read_csv('{}/ia{}/elstr{}_{}_xtb/soap418.csv'.format(location,ia,ia,ib),header=None).to_numpy()
                coors0 = np.mean(result[0][1][idx_atom],axis=0)

                idx_26b = np.squeeze(np.argwhere((bond_info[:,1]==2) | (bond_info[:,1]==6)))
                bond_info_26 = bond_info[idx_26b]
                is_broken = np.zeros(len(bond_info_26))

                for ibond in range(len(bond_info_26)): 
                    
                    idx_atom_tmp1 = bond_info_26[ibond,2]
                    idx_atom_tmp2 = bond_info_26[ibond,3]

                    for jbond in range(len(bond_info_26)):
                        if ((idx_atom_tmp1 in bond_info_26[jbond,2:4])) & (bond_info_26[jbond,1] == bond_info_26[ibond,1]) & (jbond != ibond):
                            idx_neighbor = jbond
                            if idx_atom_tmp1 == bond_info_26[jbond,2]:
                                common_at = [0,0] # which of the two atoms is the common atom
                            else:
                                common_at = [0,1]
                            break
                        elif (idx_atom_tmp2 in bond_info_26[jbond,2:4]) & (bond_info_26[jbond,1] == bond_info_26[ibond,1]) & (jbond != ibond):
                            idx_neighbor = jbond
                            if idx_atom_tmp2 == bond_info_26[jbond,2]:
                                common_at = [1,0] # which of the two atoms is the common atom
                            else:
                                common_at = [1,1]
                            break
                        else:
                            idx_neighbor = -1

                    if idx_neighbor>=0:
                        coors_atom = result[-1][1][bond_info_26[ibond,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info_26[ibond,2:4].astype(int)]
                        dist_tmp_i = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

                        coors_atom = result[-1][1][bond_info_26[idx_neighbor,2:4].astype(int)]
                        coors_atom0 = result[0][1][bond_info_26[idx_neighbor,2:4].astype(int)]
                        dist_tmp_j = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

                        if (dist_tmp_i>bond_info_26[ibond,-3]*1.5) | (dist_tmp_j>bond_info_26[idx_neighbor,-3]*1.5):
                            output = 1
                            # coors1 = np.mean(coors_atom0,axis=0)
                            # dist_core = np.sum((coors1-coors0)**2)**0.5
                            # dist_bond.append(dist_core)
                        else:
                            output = 0
                        nsoap = int(soap.shape[1]/2)
                        input = np.concatenate(([output,bond_info_26[ibond,1],bond_info_26[ibond,4],bond_info_26[ibond,5],bond_info_26[idx_neighbor,4],
                        bond_info_26[idx_neighbor,5],bond_info_26[ibond,-1],bond_info_26[idx_neighbor,-1],bond_info_26[ibond,-2],bond_info_26[idx_neighbor,-2]],
                        soap[idx_26b[ibond],nsoap*common_at[0]:nsoap*(common_at[0]+1)],
                        soap[idx_26b[ibond],nsoap*(1-common_at[0]):nsoap*(2-common_at[0])],
                        soap[idx_26b[idx_neighbor],nsoap*(1-common_at[1]):nsoap*(2-common_at[1])]))
                        # print(ib,input.shape)
                        
                        tmp.append(input)
    tmp = pd.DataFrame(np.array(tmp))
    tmp.to_csv('{}'.format(save_file),index=False)
    # type_bb = np.array(type_bb)
    # return result_oLE

In [ ]:
# def collect_data4CG(location,ia_list,save_file):
#     """
#     collect data, CG soap and output,
#     4nn input: adding SOAP together(concat, or sum)
#     """
#     class StopLookingForThings(Exception): pass

#     is_broken_all = [] 
#     tmp = []
#     is_trigger = []
#     dist_bond = []

#     # result_oLE = [] # output, length, energy

#     for ia in ia_list: # 
#         for ib in range(100):
#             # try:
#             if os.path.isfile('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib)):
#                 idx_atom, bond_info = read_info('{}/ia{}/elstr{}_{}_xtb/index_atom_interest.txt'.format(location,ia,ia,ib))
#                 result = read_mutiple_xyz_orca('{}/ia{}/elstr{}_{}_xtb/lstr_trj.xyz'.format(location,ia,ia,ib))
#                 soap = pd.read_csv('{}/ia{}/elstr{}_{}_xtb/soap418.csv'.format(location,ia,ia,ib),header=None).to_numpy()
#                 coors0 = np.mean(result[0][1][idx_atom],axis=0)

#                 idx_26b = np.squeeze(np.argwhere((bond_info[:,1]==2) | (bond_info[:,1]==6)))
#                 bond_info_26 = bond_info[idx_26b]
#                 is_broken = np.zeros(len(bond_info_26))

#                 for ibond in range(len(bond_info_26)): 
                    
#                     idx_atom_tmp1 = bond_info_26[ibond,2]
#                     idx_atom_tmp2 = bond_info_26[ibond,3]

#                     for jbond in range(len(bond_info_26)):
#                         if ((idx_atom_tmp1 in bond_info_26[jbond,2:4])) & (bond_info_26[jbond,1] == bond_info_26[ibond,1]) & (jbond != ibond):
#                             idx_neighbor = jbond
#                             if idx_atom_tmp1 == bond_info_26[jbond,2]:
#                                 common_at = [0,0] # which of the two atoms is the common atom
#                             else:
#                                 common_at = [0,1]
#                             break
#                         elif (idx_atom_tmp2 in bond_info_26[jbond,2:4]) & (bond_info_26[jbond,1] == bond_info_26[ibond,1]) & (jbond != ibond):
#                             idx_neighbor = jbond
#                             if idx_atom_tmp2 == bond_info_26[jbond,2]:
#                                 common_at = [1,0] # which of the two atoms is the common atom
#                             else:
#                                 common_at = [1,1]
#                             break
#                         else:
#                             idx_neighbor = -1

#                     if idx_neighbor>=0:
#                         coors_atom = result[-1][1][bond_info_26[ibond,2:4].astype(int)]
#                         coors_atom0 = result[0][1][bond_info_26[ibond,2:4].astype(int)]
#                         dist_tmp_i = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

#                         coors_atom = result[-1][1][bond_info_26[idx_neighbor,2:4].astype(int)]
#                         coors_atom0 = result[0][1][bond_info_26[idx_neighbor,2:4].astype(int)]
#                         dist_tmp_j = np.sum((coors_atom[0]-coors_atom[1])**2)**0.5

#                         if (dist_tmp_i>bond_info_26[ibond,-3]*1.5) | (dist_tmp_j>bond_info_26[idx_neighbor,-3]*1.5):
#                             output = 1
#                             # coors1 = np.mean(coors_atom0,axis=0)
#                             # dist_core = np.sum((coors1-coors0)**2)**0.5
#                             # dist_bond.append(dist_core)
#                         else:
#                             output = 0
#                         nsoap = int(soap.shape[1]/2)
#                         input = np.concatenate(([output,bond_info_26[ibond,1],
#                                                  bond_info_26[ibond,4],bond_info_26[ibond,5],
#                                                  bond_info_26[idx_neighbor,4],bond_info_26[idx_neighbor,5],
#                                                 #  bond_info_26[ibond,-1],bond_info_26[idx_neighbor,-1],
#                                                 #  bond_info_26[ibond,-2],bond_info_26[idx_neighbor,-2]],
#                                                 soap[idx_26b[ibond],nsoap*common_at[0]:nsoap*(common_at[0]+1)],
#                                                 soap[idx_26b[ibond],nsoap*(1-common_at[0]):nsoap*(2-common_at[0])],
#                                                 soap[idx_26b[idx_neighbor],nsoap*(1-common_at[1]):nsoap*(2-common_at[1])]))
#                         # print(ib,input.shape)
                        
#                         tmp.append(input)
#     tmp = pd.DataFrame(np.array(tmp))
#     tmp.to_csv('{}'.format(save_file),index=False)
#     # type_bb = np.array(type_bb)
#     # return result_oLE